In [5]:
%matplotlib inline
import sys
from refer import REFER
import cv2
import skimage.io as io
import numpy as np
import matplotlib.pyplot as plt
import pickle
from tqdm import tqdm
import json

from textblob import TextBlob
from textblob_aptagger import PerceptronTagger

In [6]:
def show_image(image):
    height = image.shape[0]
    width = image.shape[1]
    
    margin=50 # pixels
    dpi=100. # dots per inch

    figsize=((width+2*margin)/dpi, (height+2*margin)/dpi) # inches
    left = margin/dpi/figsize[0] #axes ratio
    bottom = margin/dpi/figsize[1]

    fig = plt.figure(figsize=figsize, dpi=dpi)
    fig.subplots_adjust(left=left, bottom=bottom, right=1.-left, top=1.-bottom)

    plt.imshow(image)
    plt.show()

In [7]:
data_root = 'data'  # contains refclef, refcoco, refcoco+, refcocog and images
dataset = 'refcoco'#'refcoco'
splitBy = 'unc'
refer = REFER(data_root, dataset, splitBy)

loading dataset refcoco into memory...
creating index...
index created.
DONE (t=8.43s)


In [8]:
ref_id=11
ref = refer.Refs[ref_id]
print(refer.Imgs[ref['image_id']]['file_name'])
#image_file_name = '../refer/data/images/mscoco/images/train2014' + '/' + refer.Imgs[ref['image_id']]['file_name']
image_file_name = '../refer/data/images/saiapr_tc-12' + '/' + refer.Imgs[ref['image_id']]['file_name']
# x,y,w,h=refer.getRefBox(ref_id)
image = cv2.imread(image_file_name)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
show_image(image)
print(ref['sentences'])
x,y,w,h=refer.getRefBox(ref_id)
print(x,y,w,h)

COCO_train2014_000000581738.jpg


error: /io/opencv/modules/imgproc/src/color.cpp:11079: error: (-215) scn == 3 || scn == 4 in function cvtColor


In [8]:
refer.Refs

{0: {'ann_id': '19135_1',
  'category_id': 60,
  'image_id': 19135,
  'ref_id': 0,
  'sent_ids': [0],
  'sentences': [{'raw': 'sky',
    'sent': 'sky',
    'sent_id': 0,
    'tokens': ['sky']}],
  'split': 'train'},
 1: {'ann_id': '19135_2',
  'category_id': 235,
  'image_id': 19135,
  'ref_id': 1,
  'sent_ids': [1],
  'sentences': [{'raw': 'statue',
    'sent': 'statue',
    'sent_id': 1,
    'tokens': ['statue']}],
  'split': 'train'},
 2: {'ann_id': '23412_4',
  'category_id': 258,
  'image_id': 23412,
  'ref_id': 2,
  'sent_ids': [2],
  'sentences': [{'raw': 'anywhere except the people',
    'sent': 'anywhere except the people',
    'sent_id': 2,
    'tokens': ['anywhere', 'except', 'the', 'people']}],
  'split': 'train'},
 3: {'ann_id': '23412_1',
  'category_id': 160,
  'image_id': 23412,
  'ref_id': 3,
  'sent_ids': [3],
  'sentences': [{'raw': 'person in front',
    'sent': 'person in front',
    'sent_id': 3,
    'tokens': ['person', 'in', 'front']}],
  'split': 'train'},
 4: 

In [10]:
filter_keywords = set([
                   'left', 'right', 'top', 'bottom', 
                   'middle', 'big', 'small', 'long', 'short', 'tall', 'huge', 'tiny',
                   'blue', 'green', 'red', 'yellow', 'pink', 'purple', 'orange', 'black', 'gray', 'white', 'brown'])

# filtered_ref_ids = set()
# filtered_ref_ids_sentences = dict()
# train_ref_ids = refer.getRefIds()
# for ref_id in tqdm(train_ref_ids):
#     ref = refer.Refs[ref_id]
#     for sentence in ref['sentences']:
#         if len(set(sentence['tokens']) & filter_keywords) > 0:
#             filtered_ref_ids.add(ref_id)
#             filtered_ref_ids_sentences[ref_id] = sentence
# len(filtered_ref_ids)

In [5]:
filtered_ref_ids_sentences

{0: {'raw': 'blue shirt',
  'sent': 'blue shirt',
  'sent_id': 2,
  'tokens': ['blue', 'shirt']},
 1: {'raw': 'woman greyshirt right',
  'sent': 'woman greyshirt right',
  'sent_id': 5,
  'tokens': ['woman', 'greyshirt', 'right']},
 2: {'raw': 'middle standing up',
  'sent': 'middle standing up',
  'sent_id': 8,
  'tokens': ['middle', 'standing', 'up']},
 3: {'raw': 'woman sitting on right',
  'sent': 'woman sitting on right',
  'sent_id': 11,
  'tokens': ['woman', 'sitting', 'on', 'right']},
 4: {'raw': 'left person',
  'sent': 'left person',
  'sent_id': 13,
  'tokens': ['left', 'person']},
 6: {'raw': 'left urinal',
  'sent': 'left urinal',
  'sent_id': 20,
  'tokens': ['left', 'urinal']},
 7: {'raw': 'right urinal',
  'sent': 'right urinal',
  'sent_id': 23,
  'tokens': ['right', 'urinal']},
 8: {'raw': 'right most vase',
  'sent': 'right most vase',
  'sent_id': 25,
  'tokens': ['right', 'most', 'vase']},
 9: {'raw': 'middle', 'sent': 'middle', 'sent_id': 27, 'tokens': ['middle']}

In [15]:
boxes = dict()
image_ids = dict()
image_file_names = dict()
sentences = dict()

train_ref_ids = refer.getRefIds()

for ref_id in tqdm(train_ref_ids):
    ref = refer.Refs[ref_id]
    x,y,w,h=refer.getRefBox(ref_id)
    boxes[ref_id] = (x,y,w,h)
    image_ids[ref_id] = ref['image_id']
    image_file_name = '../refer/data/images/mscoco/images/train2014' + '/' + refer.Imgs[ref['image_id']]['file_name']
    #image_file_name = '../refer/data/images/mscoco/images/saiapr_tc-12' + '/' + refer.Imgs[ref['image_id']]['file_name']
    image_file_names[ref_id] = image_file_name
    ref_sentences = []
    for sentence in ref['sentences']:
        sent = sentence['sent']
        blob = TextBlob(sent, pos_tagger=PerceptronTagger())
        tags = blob.tags
        subject = None
        for token, tag in tags:
            if token not in filter_keywords and tag == 'NN':
                subject = token
                break

        ref_sentences.append({'subject': subject, 'sentence': sentence, 'tags': tags})
    sentences[ref_id] = ref_sentences
        
my_refer_data_all_sentences = {'boxes': boxes, 
                               'image_ids': image_ids,
                               'image_file_names': image_file_names,
                               'sentences': sentences}

with open('my_refer_data_all_sentences.pickle', 'wb') as file:
    pickle.dump(my_refer_data_all_sentences, file)

100%|██████████| 50000/50000 [5:39:22<00:00,  2.46it/s]  


In [17]:
len(my_refer_data_all_sentences['boxes'])

50000

In [ ]:
with open('my_refer_data.pickle', 'rb') as file:
    my_refer_data = pickle.load(file)

In [12]:
x=3

In [16]:
x

3

In [17]:
len(my_refer_data['ref_ids'])

40062

In [37]:
train_ref_ids = refer.getRefIds(split='train')
with open('train_ref_ids.pickle', 'wb') as file:
    pickle.dump(train_ref_ids, file)

In [5]:
val_ref_ids = refer.getRefIds(split='val')
with open('val_ref_ids.pickle', 'wb') as file:
    pickle.dump(val_ref_ids, file)

In [8]:
with open('filter_keywords.pickle', 'wb') as file:
    pickle.dump(filter_keywords, file)

In [26]:
# train_ref_ids = refer.getRefIds()

# ref_sentences_and_pos = []

# for ref_id in tqdm(train_ref_ids):
#     ref = refer.Refs[ref_id]
#     for sentence in ref['sentences']:
#         blob = TextBlob(sentence['sent'], pos_tagger=PerceptronTagger())
#         tags = blob.tags
#         ref_sentences_and_pos.append({'ref_id': ref_id, 
#                                       'sent_id': sentence['sent_id'], 
#                                       'tokens': sentence['tokens'],
#                                       'sent': sentence['sent'],
#                                       'tags': tags})

# with open('ref_sentences_and_pos.pickle', 'wb') as file:
#     pickle.dump(ref_sentences_and_pos, file)

100%|██████████| 50000/50000 [8:06:07<00:00,  1.71it/s]  


In [34]:
adjectives = {}

for sentence in tqdm(ref_sentences_and_pos):
    for token, tag in sentence['tags']:
        if tag == 'JJ':
            if token in adjectives:
                adjectives[token] += 1
            else:
                adjectives[token] = 1

100%|██████████| 142210/142210 [00:00<00:00, 198098.88it/s]


In [36]:
top_adjectives = [(k, adjectives[k]) for k in sorted(adjectives, key=adjectives.get, reverse=True)]

In [45]:
top_adjectives_10_plus = [x for x in top_adjectives if x[1]>9]

In [46]:
top_adjectives_10_plus

[('right', 8551),
 ('white', 7530),
 ('black', 5444),
 ('red', 4473),
 ('blue', 4427),
 ('top', 3718),
 ('middle', 3556),
 ('second', 2713),
 ('front', 2559),
 ('green', 2412),
 ('bottom', 1950),
 ('left', 1939),
 ('yellow', 1280),
 ('big', 1039),
 ('brown', 956),
 ('third', 952),
 ('gray', 929),
 ('upper', 656),
 ('dark', 607),
 ('little', 581),
 ('back', 577),
 ('first', 543),
 ('small', 459),
 ('partial', 433),
 ('elephant', 400),
 ('next', 389),
 ('main', 370),
 ('old', 339),
 ('light', 337),
 ('hot', 316),
 ('lady', 304),
 ('full', 268),
 ('purple', 244),
 ('large', 232),
 ('empty', 199),
 ('tall', 171),
 ('whole', 165),
 ('other', 159),
 ('couch', 156),
 ('striped', 153),
 ('fourth', 151),
 ('tan', 150),
 ('last', 149),
 ('open', 145),
 ('teddy', 143),
 ('visible', 140),
 ('sandwich', 131),
 ('short', 121),
 ('plaid', 114),
 ('blurry', 114),
 ('long', 113),
 ('urinal', 106),
 ('sheep', 104),
 ('orange', 94),
 ('bear', 88),
 ('asian', 76),
 ('fat', 73),
 ('animal', 69),
 ('wooden'

In [7]:
len(refer.getRefIds(split='test'))

3785

In [5]:
len(refer.getRefIds())

50000

In [4]:
abc={'a':1,'b':2}
abc.values()

dict_values([1, 2])